In [32]:
%load_ext autoreload 
%autoreload 2

%aimport -pandas
%aimport -scipy.stats
%aimport -numpy
%aimport -matplotlib.pyplot


import default_calcs as default
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas
import collateral_waterfall as cw
import prepayment_calcs as pc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
initial_balance = 100e6    # 100 million
net_coupon      = 0.10     # 10%
gross_coupon    = 0.1065   # 10.65%
maturity        = 360      # 30 years => monthly
servicing       = 0.0025   # servicing = 25bps

In [21]:
psa_100 = cw.create_waterfall(original_balance=initial_balance,
                             pass_thru_cpn=net_coupon,
                             wac=gross_coupon,
                             wam=maturity,
                             psa_speed=1,
                             servicing=servicing)
psa_175 = cw.create_waterfall(original_balance=initial_balance,
                             pass_thru_cpn=net_coupon,
                             wac=gross_coupon,
                             wam=maturity,
                             psa_speed=1.75,
                             servicing=servicing)
psa_400 = cw.create_waterfall(original_balance=initial_balance,
                             pass_thru_cpn=net_coupon,
                             wac=gross_coupon,
                             wam=maturity,
                             psa_speed=4,
                             servicing=servicing)

In [22]:
print('PSA\tAverage Life')
print('100%:\t{0:.2f}'.format(weighted_average_life(psa_100)/12))
print('175%:\t{0:.2f}'.format(weighted_average_life(psa_175)/12))
print('400%:\t{0:.2f}'.format(weighted_average_life(psa_400)/12))


PSA	Average Life
100%:	12.40
175%:	8.93
400%:	4.75


In [31]:
%matplotlib
frames = [psa_100,psa_175,psa_400]

fig, axes = plt.subplots(3,2)
for i in range(3):
    for j in range(2):
        if j == 0:
            axes[i,j].fill_between(frames[i].index-1,
                                  frames[i].beginning_balance)
        else:
            axes[i,j].stackplot(frames[i].index-1, 
                          frames[i].scheduled_principal,
                          frames[i].prepayments,
                          frames[i].net_interest,
                          frames[i].servicing)
            
plt.tight_layout(True)
plt.show()

Using matplotlib backend: MacOSX
